# LLM setup notebook

In [ ]:
!pip install ollama -q > /dev/null

In [ ]:
# Importing all the necessary libraries
import pandas as pd
import zipfile
import os
import json
import ollama
import re
from collections import Counter
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Code to connect all the files in within the ZIP file into one DF

In [ ]:
# Path to the ZIP file
zip_file_path = '/content/drive/My Drive/Files/file.zip'

# Temporary folder to extract JSON files
extract_folder = '/content/json_files'

# Extract all files from the ZIP
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

# Initialize a list to store data
dataframes = []

# Iterate over each JSON file in the extracted folder
for file_name in os.listdir(extract_folder):
    if file_name.endswith('.json'):  # Check if it's a JSON file
        file_path = os.path.join(extract_folder, file_name)
        try:
            # Read the JSON file
            with open(file_path, 'r', encoding='utf-8') as file:
                data = json.load(file)

                df_temp = pd.DataFrame(data)
                dataframes.append(df_temp)
        except Exception as e:
            print(f"Error reading {file_name}: {e}")

# Combine all DataFrames into one named df
df = pd.concat(dataframes, ignore_index=True)

## Data Preprocessing Before LLM

In this step, we preprocess the data to ensure it is ready for the LLM. We remove rows with missing or insufficient information, as well as those that do not align with the project's objectives, ensuring that only relevant and complete data is passed for further processing.

In [ ]:
 # Calculate word counts for each row in the 'HTML_Text' column
df['word_count'] = df['HTML_Text'].apply(lambda x: len(x.split()))

# Generate statistics on the word counts
stats = df['word_count'].describe()

# Print the statistics
print(stats)

count    1494.000000
mean      629.763052
std       344.463381
min        66.000000
25%       413.250000
50%       670.000000
75%       826.000000
max      4080.000000
Name: word_count, dtype: float64


### We remove the HTML texts where the word count is less than 400 as those do not give use enought info to work with.

In [ ]:
# Define a threshold for minimum word count
min_word_count = 400

# Filter the original dataframe in place, only keeping rows meeting the word count requirement
df = df[df['HTML_Text'].apply(lambda x: len(x.split()) > min_word_count)].copy()

df.shape

(1127, 9)

### Removing additional letters from city names. For example ''Aalborg C'' would become ''Aalborg''

In [ ]:
# Function to city area indications
def clean_trailing_indicators(area):
    if pd.isna(area):
        return None
    # Remove things like 'o', 'c', 'SØ', 'N', 'K'
    area = re.sub(r'\s+[a-zA-ZÆØÅæøå]+$', '', area.strip())
    return area

# Apply the cleaning function to the 'Area' column
df['Area'] = df['Area'].apply(clean_trailing_indicators)

### Some ads have more than one city listed as an option. We only keep the city that was listed as the first option

In [ ]:
# Simplified function to keep the first city
def keep_first_place_simple(area):
    if pd.isna(area):
        return None
    # Split by simpler delimiters
    delimiters = [',', '/', ' og', ' eller', ' or', ' and']
    for delim in delimiters:
        if delim in area:
            # Keep only the first part
            area = area.split(delim)[0].strip()
            break
    # Return cleaned area if it is meaningful
    return area if len(area) > 2 else None

# Apply the simplified cleaning function
df['Area'] = df['Area'].apply(keep_first_place_simple)

### Mapping the 3 cities that have different name in English vs Danish

In [ ]:
# Mapping of Danish to English city names
city_name_mapping = {
    'København': 'Copenhagen',
    'Århus': 'Aarhus',
    'Helsingør': 'Elsinore'
}

# Replace Danish names with their English equivalents
df['Area'] = df['Area'].replace(city_name_mapping)

### A list of cities we want to look at. If the area is not one of the ones listed, it is removed. These are the 30 biggest cities in Denmark

In [ ]:
# List of 30 biggest cities in Denmark
cities = [
    "Copenhagen", "Aarhus", "Odense", "Aalborg", "Esbjerg", "Randers", "Kolding", "Horsens", "Vejle", "Roskilde",
    "Herning", "Hørsholm", "Silkeborg", "Næstved", "Fredericia", "Viborg", "Køge", "Holstebro", "Taastrup", "Slagelse",
    "Hillerød", "Sønderborg", "Svendborg", "Hjørring", "Holbæk", "Frederikshavn", "Nørresundby", "Ringsted", "Haderslev",
    "Skive", "Ølstykke-Stenløse", "Nykøbing Falster", "Greve Strand", "Kalundborg", "Ballerup", "Rødovre", "Lyngby",
    "Albertslund", "Hvidovre", "Glostrup", "Ishøj", "Birkerød", "Farum", "Frederikssund", "Brøndby Strand",
    "Skanderborg", "Hedensted", "Frederiksværk", "Lillerød", "Solrød Strand"
]

In [ ]:
# Filter the DataFrame to only include rows where Area is in the cities list
df = df[df['Area'].isin(cities)]

# Display the filtered DataFrame
print("Filtered DataFrame:")
print(df.shape)

# This is a sample data. Only for a few months

Filtered DataFrame:
(620, 9)


In [ ]:
# Install Ollama
!sudo apt-get install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpci3 pci.ids
The following NEW packages will be installed:
  libpci3 pci.ids pciutils
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 343 kB of archives.
After this operation, 1,581 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 pci.ids all 0.0~2022.01.22-1 [251 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpci3 amd64 1:3.7.0-6 [28.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 pciutils amd64 1:3.7.0-6 [63.6 kB]
Fetched 343 kB in 0s (1,123 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 3.)
debconf: falling back to frontend: Readline
debconf: unable to initial

In [ ]:
# Sets up environment variables and starts the Ollama server
import os
import threading
import subprocess

def start_ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

ollama_thread = threading.Thread(target=start_ollama)
ollama_thread.start()

In [ ]:
!ollama run llama3.1:8b

In [ ]:
SYSTEM_PROMPT = """

Extract relevant skills from a job listing, separating them into **primary IT-related technical skills** and **secondary (soft/general) skills**.

### Primary Skills
Focus on IT-specific technical skills such as:
- Programming languages, frameworks, tools, platforms, and technologies.
- Examples: Python, React, Docker, AWS, SQL, Kubernetes.

### Secondary Skills
Focus on non-technical competencies, including:
- Communication, leadership, problem-solving, teamwork, time management.

---

### Rules:
- Categorize skills into **primary** (IT-related) and **secondary** (soft skills).
- Split compound terms into separate skills (e.g., "Python and Java" → ["Python", "Java"]).
- Exclude vague, repetitive, or non-actionable terms.
- Translate all extracted skills into **English** if the input text is in Danish.
- Ensure outputs are clear, specific, and concise.


---

### Output Format:
```json
{
  "skills": {
    "primary": ["Skill 1", "Skill 2", "..."],
    "secondary": ["Skill A", "Skill B", "..."]
  }
}

---

Example:

Job: "Looking for Python, Docker, AWS, and strong communication skills."

Output:
{
    "skills": {
    "primary": ["Python", "Docker", "AWS"],
    "secondary": ["communication"]
  }
}

"""

In [ ]:
# Sets up environment variables and starts the Ollama server
import os
import threading
import subprocess

def start_ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

ollama_thread = threading.Thread(target=start_ollama)
ollama_thread.start()

In [ ]:
import pandas as pd
import json

# Function to extract skills (primary and secondary) from job listings
def extract_skills(html_text, row_id):
    prompt = f"""
    Extract the relevant skills from the following job listing, categorizing them into **primary (IT-related technical skills)** and **secondary (soft/general skills)**.

    ### Skills Categories:
    1. **Primary Skills:** IT-related technical skills (e.g., Python, AWS, Kubernetes, Docker, SQL, React, etc.).
    2. **Secondary Skills:** Soft skills and traits (e.g., Communication, Teamwork, Problem-Solving, Leadership).

    ### Rules:
    - Split combined skills into distinct terms (e.g., "Python and Java" → ["Python", "Java"]).
    - Focus on actionable, specific, and clearly stated skills.
    - Outputs must be in **clear and concise English only**, regardless of the input language.
    - If no skills are present, return empty lists for both categories.
    - Translate all extracted skills into **English** if the input text is in Danish.


    Provide the results in this JSON format:
    {{
      "skills": {{
        "primary": ["Skill 1", "Skill 2", "..."],
        "secondary": ["Skill A", "Skill B", "..."]
      }}
    }}

    Job Listing Text:
    {html_text}
    """
    try:
        response = ollama.chat(
            model='llama3.1:8b',
            messages=[
                {'role': 'system', 'content': SYSTEM_PROMPT},
                {'role': 'user', 'content': prompt},
            ],
            format='json',
            options={"temperature": 0.1}
        )
        return json.loads(response['message']['content'])
    except (json.JSONDecodeError, KeyError) as e:
        print(f"Error parsing JSON for Row {row_id}: {e}")
        return {"skills": {"primary": [], "secondary": []}}

# Function to refine extracted skills
def refine_skills(extracted_skills, row_id):
    refinement_prompt = f"""
    Refine the following extracted skills, ensuring accuracy and clear categorization:

    Extracted Skills:
    {json.dumps(extracted_skills, indent=2)}

    ### Refined Categories:
    1. **Primary Skills:** IT-related technical skills only.
    2. **Secondary Skills:** Clearly defined soft skills.

    ### Rules:
    - Keep skills actionable, specific, and IT-relevant for "primary."
    - Use concise and clear English.
    - Outputs must be in **clear and concise English.**
    - Translate all extracted skills into **English** if the input text is in Danish.

    Provide the results in this JSON format:
    {{
      "skills": {{
        "primary": ["Skill 1", "Skill 2", "..."],
        "secondary": ["Skill A", "Skill B", "..."]
      }}
    }}
    """
    try:
        response = ollama.chat(
            model='llama3.1:8b',
            messages=[
                {'role': 'system', 'content': SYSTEM_PROMPT},
                {'role': 'user', 'content': refinement_prompt},
            ],
            format='json',
            options={"temperature": 0.1}
        )
        return json.loads(response['message']['content'])
    except (json.JSONDecodeError, KeyError) as e:
        print(f"Error refining skills for Row {row_id}: {e}")
        return extracted_skills

# Initialize the DataFrame with the columns for Primary_Skills and Secondary_Skills
df['Primary_Skills'] = None
df['Secondary_Skills'] = None

# Process job listings and add extracted skills to the DataFrame
for i, html_text in enumerate(df['HTML_Text'][:5]):
    # Extract skills
    extracted = extract_skills(html_text, row_id=i + 1)
    print(f"Extracted Skills from Row {i + 1}: {extracted}")

    # Refine extracted skills
    refined = refine_skills(extracted, row_id=i + 1)
    print(f"Refined Skills for Row {i + 1}: {refined}")

    # Add the refined skills as new columns to the DataFrame
    df.at[i, 'Primary_Skills'] = refined['skills']['primary']
    df.at[i, 'Secondary_Skills'] = refined['skills']['secondary']

# Display the updated DataFrame
print(df[['Primary_Skills', 'Secondary_Skills']])

# Optionally, save the updated DataFrame to a CSV or JSON file
output_file = 'updated_job_listings_with_skills.csv'
try:
    df.to_csv(output_file, index=False, encoding='utf-8')
    print(f"Updated job listings saved to {output_file}.")
except IOError as e:
    print(f"Error saving file: {e}")

Extracted Skills from Row 1: {'skills': {'primary': ['Python', 'AWS', 'Kubernetes', 'Docker', 'SQL', 'React', 'Accenture Technology', 'IT projects', 'State of the art technology', 'Security', 'Information Security', 'IT Risk Management', 'Enterprise security architecture', 'Data protection and privacy', 'Business continuity strategy', 'Identity and Access management', 'Infrastructure security', 'SOA and middleware security', 'Governance, Risk and Compliance', 'Biometrics and smart identity solutions'], 'secondary': ['Communication', 'Teamwork', 'Problem-Solving', 'Leadership', 'Marketing and networking', 'Relationship-building', 'Collaboration', 'Project management', 'People management', 'Personal leadership skills', 'Quality assurance processes']}}
Refined Skills for Row 1: {'skills': {'primary': ['Python', 'AWS', 'Kubernetes', 'Docker', 'SQL', 'React'], 'secondary': ['Communication', 'Teamwork', 'Problem-Solving', 'Leadership', 'Project management', 'People management', 'Collaboratio

In [ ]:
df.head(5)

,Title,URL,Area,Category_Job_ID,Category_Job,Published,Job Link,HTML_Text,word_count,Primary_Skills,Secondary_Skills
0,Consultants to Information Security and IT Ris...,https://www.jobindex.dk/vis-job/r6572791,Copenhagen,4,IT maintenance and Support,2016-01-10,https://www.jobindexarkiv.dk/cgi/showarchive.c...,Lignende jobannoncer Job i Storkøbenhavn kateg...,815,"[Python, AWS, Kubernetes, Docker, SQL, React]","[Communication, Teamwork, Problem-Solving, Lea..."
1,Experienced Infrastructure Consultant for IT S...,https://www.jobindex.dk/vis-job/r6572792,Copenhagen,4,IT maintenance and Support,2016-01-10,https://www.jobindexarkiv.dk/cgi/showarchive.c...,Lignende jobannoncer Job i Storkøbenhavn kateg...,610,"[Python, AWS, Kubernetes, Docker, SQL, React]","[Communication, Teamwork, Problem-Solving, Lea..."
2,Head of IT Audit,https://www.jobindex.dk/vis-job/r6572793,Copenhagen,3,IT Management,2016-01-10,https://www.jobindexarkiv.dk/cgi/showarchive.c...,Lignende jobannoncer Job i Storkøbenhavn kateg...,667,"[IT Audit, Cyber Security, Risk Management, IT...","[Leadership, Teamwork, Communication, Problem-..."
3,Oracle Fusion Middleware Consultant,https://www.jobindex.dk/vis-job/r6572750,Copenhagen,1,System development and programming,2016-01-10,https://www.jobindexarkiv.dk/cgi/showarchive.c...,Lignende jobannoncer Job i Storkøbenhavn kateg...,526,"[Oracle Fusion Middleware, Python, Java, SQL, ...","[Communication, Teamwork, Problem-Solving, Lea..."
4,Head of Accounting,https://www.jobindex.dk/vis-job/r6572778,Copenhagen,1,System development and programming,2016-01-10,https://www.jobindexarkiv.dk/cgi/showarchive.c...,Lignende jobannoncer Job i Storkøbenhavn kateg...,625,[Navision],"[Leadership, Management, Communication, Proble..."
